In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout ,BatchNormalization
from keras import backend as K
from tqdm import tqdm_notebook,tnrange

# 准备
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

train_df = pd.read_csv("/home/zhangs/lyc/salt/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("/home/zhangs/lyc/salt/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]#将生成id不在train中的样本id集合
train_df["images"] = [np.array(load_img("/home/zhangs/lyc/salt/train/images/{}.png".format(idx), grayscale=True))/ 255 for idx in tqdm_notebook(train_df.index)]
train_df["masks"] = [np.array(load_img("/home/zhangs/lyc/salt/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)
# 将深度信息放入训练图像
MAX_DEPTH = max(train_df["z"])
print('**** Max depth in train set is :'+str(MAX_DEPTH))
train_df["depth"] = [np.ones_like(train_df.loc[i]["images"]) * train_df.loc[i]["z"] / MAX_DEPTH
                     for i in tqdm_notebook(train_df.index)]

# Image in layer1 + depth in layer2
train_df["images_d"] = [np.dstack((train_df["images"][i],train_df["depth"][i])) for i in tqdm_notebook(train_df.index)]
train_df["images_d"][0].shape
# Free up some RAM
del depths_df
del train_df["images"]
# Sanity check
train_df.head()

In [3]:

from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout ,BatchNormalization

help(BatchNormalization)

/home/zhangs/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Help on class BatchNormalization in module keras.layers.normalization:

class BatchNormalization(keras.engine.base_layer.Layer)
 |  Batch normalization layer (Ioffe and Szegedy, 2014).
 |  
 |  Normalize the activations of the previous layer at each batch,
 |  i.e. applies a transformation that maintains the mean activation
 |  close to 0 and the activation standard deviation close to 1.
 |  
 |  # Arguments
 |      axis: Integer, the axis that should be normalized
 |          (typically the features axis).
 |          For instance, after a `Conv2D` layer with
 |          `data_format="channels_first"`,
 |          set `axis=1` in `BatchNormalization`.
 |      momentum: Momentum for the moving mean and the moving variance.
 |      epsilon: Small float added to variance to avoid dividing by zero.
 |      center: If True, add offset of `beta` to normalized tensor.
 |          If False, `beta` is ignored.
 |      scale: If True, multiply by `gamma`.
 |          If False, `gamma` is not us

In [ ]:
ids_train, ids_valid = train_test_split(
    train_df.index.values,
    test_size=0.05, stratify=train_df.coverage_class, random_state=1337)
my_test_pd = train_df.loc[train_df.index.isin(ids_valid)]
train_df = train_df.loc[train_df.index.isin(ids_train)]

In [ ]:
print(my_test_pd.shape)
print(train_df.shape)


# Cut the train and valid set ，use K-flods

In [ ]:
K_flods = 5
from sklearn.model_selection import StratifiedKFold
X = train_df.index.values
y = train_df.coverage_class
skf = StratifiedKFold(n_splits=K_flods,random_state=1337)
skf.get_n_splits(X, y)
print(skf)
ids_train,ids_valid,x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test=[[] for x in range(10)]
X_whole = np.array(train_df.images_d.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 2)
y_whole = np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)
for i,[train_index, test_index] in enumerate(skf.split(X, y)):
    print("the %dth flod:"%i)
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    ids_train.append(X[train_index])
    ids_valid.append(X[test_index])
    #
    x_train.append(X_whole[train_index])
    x_valid.append(X_whole[test_index])
    #
    y_train.append(y_whole[train_index])
    y_valid.append(y_whole[test_index]) 
    #
    cov_train.append(train_df.coverage.values[train_index]) 
    cov_test.append(train_df.coverage.values[test_index])
    #
    depth_train.append(train_df.z.values[train_index]) 
    depth_test.append(train_df.z.values[test_index]) 
del X_whole,y_whole
print(len(x_train))
print(x_train[0].shape)

In [ ]:
# 画各个flods的salt分布图，检验k-flods是否正确
def plot_flods_coverage(cov,flods_num=5,mode='train'):
    fig, axs = plt.subplots(1, flods_num+1, figsize=(15,5))
    sns.distplot(train_df.coverage, kde=False, ax=axs[0])
    for i in range(1,flods_num+1):
        sns.distplot(cov[i-1], bins=10, kde=False, ax=axs[i])
        axs[i].set_xlabel("Coverage of k%d"%(i-1))
    plt.suptitle("Salt coverage of k-flods "+mode)
    axs[0].set_xlabel("Coverage")
    plt.show()
    

In [ ]:
plot_flods_coverage(cov_train,flods_num=5,mode='train')

In [ ]:
plot_flods_coverage(cov_test,flods_num=5,mode='test')

In [ ]:
# sns.distplot(train_df.z, label="Train")
# sns.distplot(test_df.z, label="Test")
# plt.legend()
# plt.title("Depth distribution")
# plt.show()

In [ ]:
# max_images = 60
# grid_width = 15
# grid_height = int(max_images / grid_width)
# fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
# for i, idx in enumerate(train_df.index[:max_images]):
#     img = train_df.loc[idx].images
#     mask = train_df.loc[idx].masks
#     ax = axs[int(i / grid_width), i % grid_width]
#     ax.imshow(img, cmap="Greys")
#     ax.imshow(mask, alpha=0.3, cmap="Greens")
#     ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
#     ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
#     ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
#     ax.set_yticklabels([])
#     ax.set_xticklabels([])
# plt.suptitle("Green: salt. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")
# plt.show()

In [ ]:
# ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
#     train_df.index.values,
#     np.array(train_df.images_d.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 2), 
#     np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
#     train_df.coverage.values,
#     train_df.z.values,
#     test_size=0.2, stratify=train_df.coverage_class, random_state=1337)

In [ ]:
from skimage.util import pad
fig, axs = plt.subplots(1, 4, figsize=(15,5))
axs[0].imshow(x_train[0,:,:,0],  cmap="Greys")
axs[1].imshow(x_train[1,:,:,0],  cmap="Greys")
 
xpad1=pad(resize(x_train[0,:,:,0], (101*2, 101*2), mode='constant', preserve_range=True),27,'reflect')
xpad2=pad(resize(x_train[1,:,:,0], (101*2, 101*2), mode='constant', preserve_range=True),27,'reflect')
print(xpad1.shape)
axs[2].imshow(xpad1,  cmap="Greys")
axs[3].imshow(xpad2,  cmap="Greys")
plt.show()

In [ ]:
# tmp_img = np.zeros((img_size_target, img_size_target), dtype=train_df.images.loc[ids_train[10]].dtype)
# tmp_img[:img_size_ori, :img_size_ori] = train_df.images.loc[ids_train[10]]
# fix, axs = plt.subplots(1, 2, figsize=(15,5))
# axs[0].imshow(tmp_img, cmap="Greys")
# axs[0].set_title("Original image")
# axs[1].imshow(x_train[10].squeeze(), cmap="Greys")
# axs[1].set_title("Scaled image")

# Data argumantant

In [ ]:
for i in range(K_flods):
    x_l_r_flip = [np.fliplr(x) for x in x_train[i]]
    y_l_r_flip = [np.fliplr(x) for x in y_train[i]]
    # x_u_d_flip = [np.flipud(x) for x in x_train]
    # y_u_d_flip = [np.flipud(x) for x in y_train]
    # x_t = [np.reshape(np.transpose(x),(img_size_target,img_size_target,2)) for x in x_train]
    # y_t = [np.reshape(np.transpose(x),(img_size_target,img_size_target,1)) for x in y_train]
    # x_train = np.append(x_train, x_u_d_flip, axis=0)
    # y_train = np.append(y_train, y_u_d_flip, axis=0)
    # x_train = np.append(x_train, x_t, axis=0)
    # y_train = np.append(y_train, y_t, axis=0)
    x_train[i] = np.append(x_train[i], x_l_r_flip, axis=0)
    y_train[i] = np.append(y_train[i], y_l_r_flip, axis=0)

In [ ]:
print(y_train[0].shape)

## 对比度增强

In [ ]:
# from skimage import exposure
# contrast = [np.dstack([np.array(exposure.rescale_intensity(
#     img[:,:,0],in_range=(np.percentile(img[:,:,0], (1, 99))[0],np.percentile(img[:,:,0], (1, 99))[1]))),img[:,:,1]]) 
#             for img in tqdm_notebook(x_train)]
# contrast = np.array(contrast)
# x_train = np.append(x_train, contrast, axis=0)
# y_train = np.append(y_train, y_train, axis=0)
# x_train = np.append(x_train, x_l_r_flip, axis=0)
# y_train = np.append(y_train, y_l_r_flip, axis=0)
# print(x_train.shape)
# print(y_train.shape)
# del contrast


In [ ]:
plt.subplot(1,2,1)
plt.imshow(x_train[110,:,:,0], cmap='binary')
plt.title('Original Image')
plt.subplot(1,2,2)    
plt.imshow(contrast[110,:,:,0], cmap='binary')
plt.title('Contrast stretching')
print(contrast.shape)
print(contrast[110,60:64,60:64,1])
print(x_train[110,60:64,60:64,1])

In [ ]:
import random
def random_shiff(x_train,y_train):
    length = x_train.shape[0]
    len_list = list(range(length))
    shiff_index = np.random.shuffle(len_list)
    return x_train[shiff_index:,:,:],y_train[shiff_index:,:,:]
x_train,y_train = random_shiff(x_train,y_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(np.array(x_t).shape)

In [ ]:
fix, axs = plt.subplots(2, 2, figsize=(15,5))
print(x_train.shape)
axs[0][0].imshow(x_train[10,:,:,0], cmap="Greys")
axs[0][1].imshow(y_train[10,:,:,0], cmap="Greys")
axs[1][0].imshow((np.fliplr(x_train[10,:,:,0])), cmap="Greys")
axs[1][1].imshow((np.fliplr(y_train[10,:,:,0])), cmap="Greys")

In [ ]:
# from PIL import Image, ImageEnhance, ImageOps, ImageFile
# def randomRotation(image, mode=Image.BICUBIC):
#     """
#      对图像进行随机任意角度(0~360度)旋转
#     :param mode 邻近插值,双线性插值,双三次B样条插值(default)
#     :param image PIL的图像image
#     :return: 旋转转之后的图像
#     """
#     random_angle = np.random.randint(1, 360)
#     return image.rotate(random_angle, mode)

In [ ]:
# fig, axs = plt.subplots(2, 10, figsize=(15,3))
# for i in range(10):
#     axs[0][i].imshow(x_train[i].squeeze(), cmap="Greys")
#     axs[0][i].imshow(y_train[i].squeeze(), cmap="Greens", alpha=0.3)
#     axs[1][i].imshow(x_train[int(len(x_train)/2 + i)].squeeze(), cmap="Greys")
#     axs[1][i].imshow(y_train[int(len(y_train)/2 + i)].squeeze(), cmap="Greens", alpha=0.3)
# fig.suptitle("Top row: original images, bottom row: augmented images")

In [ ]:
x_train[0][..., :1].shape
x_train[0][:, :DPT_SIZE, :DPT_SIZE, 1:]

# Model build

In [ ]:
def mean_iou(Y_true, Y_pred, score_thres=0.5):
    """Compute mean(IoU) metric
    IoU = intersection / union
    
    For each (mask)threshold in provided range:
     - convert probability mask to boolean mask based on given threshold
     - score the mask 1 if(IoU > score_threshold(0.5))
    Take the mean of the scoress

    https://www.tensorflow.org/api_docs/python/tf/metrics/mean_iou
    """
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        Y_pred_bool = tf.to_int32(Y_pred > t) # boolean mask by threshold
        score, update_op = tf.metrics.mean_iou(Y_true, Y_pred_bool, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            score = tf.identity(score) #!! use identity to transform score to tensor
        prec.append(score) 
        
    return K.mean(K.stack(prec), axis=0)

def conv_block(m, ch_dim, acti='relu', bn=True, res=True, drop=0):
    """CNN block"""
    n = Conv2D(ch_dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(drop)(n) if drop else n
    n = Conv2D(ch_dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return concatenate([m, n]) if res else n

In [ ]:
dpt = 5
DPT_SIZE = int(img_size_target/pow(2,dpt))

In [ ]:

def build_model(start_neurons):
    inputs = Input(shape=(img_size_target, img_size_target, 1), name='img')
    input_depth = Input(shape=(DPT_SIZE, DPT_SIZE, 1), name='depth')
    # 128 -> 64
    conv1 = conv_block(inputs, start_neurons*1, acti='relu', bn=True, res=True, drop=0)
    pool1 = MaxPooling2D((2, 2))(conv1)

    # 64 -> 32
    conv2 = conv_block(pool1, start_neurons*2, acti='relu', bn=True, res=True, drop=0)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # 32 -> 16
    conv3 = conv_block(pool2, start_neurons*4, acti='relu', bn=True, res=True, drop=0)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # 16 -> 8
    conv4 = conv_block(pool3, start_neurons*8, acti='relu', bn=True, res=True, drop=0)
    pool4 = MaxPooling2D((2, 2))(conv4)
    
    # 8 -> 4
    conv5 = conv_block(pool4, start_neurons*16, acti='relu', bn=True, res=True, drop=0)
    pool5 = MaxPooling2D((2, 2))(conv5)
    
    # Middle
    convm = conv_block(pool5, start_neurons*32, acti='relu', bn=True, res=True, drop=0.5)
    convm = concatenate([convm, input_depth])
    
    # 4 -> 8
    deconv5 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv5 = concatenate([deconv5, conv5])
    uconv5 = conv_block(uconv5, start_neurons * 16, acti='relu', bn=True, res=True, drop=0)
    
    # 8 -> 16
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv5)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = conv_block(uconv4, start_neurons*8, acti='relu', bn=True, res=True, drop=0)

    # 16 -> 32
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = conv_block(uconv3, start_neurons*4, acti='relu', bn=True, res=True, drop=0)

    # 32 -> 64
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_block(uconv2, start_neurons*2, acti='relu', bn=True, res=True, drop=0)

    # 64 -> 128
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_block(uconv1, start_neurons*1, acti='relu', bn=True, res=True, drop=0)

    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return Model(inputs=[inputs,input_depth], outputs=output_layer)

# output_layer = build_model(input_layer[:,:,:,:1], 16,input_layer[:,:8,:8,1:])

In [ ]:
history_all = []
fig, axs = plt.subplots(K_flods, 3, figsize=(15,5))
for i in range(K_flods): 
    model = build_model(start_neurons=32)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy",mean_iou])
#     model.summary()
    early_stopping = EarlyStopping(patience=6, verbose=1)
    model_checkpoint = ModelCheckpoint("trained_models/%dth_flod.model"%i, save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

    epochs = 200
    batch_size = 32

    history = model.fit({'img': x_train[i][..., :1], 
                         'depth': x_train[i][:, 60:60+DPT_SIZE, 60:60+DPT_SIZE, 1:]},
                        y_train[i],
                        validation_data=({'img': x_valid[i][..., :1], 
                                          'depth': x_valid[i][:, 60:60+DPT_SIZE, 60:60+DPT_SIZE, 1:]},
                        y_valid[i]), 
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping, model_checkpoint, reduce_lr])
    history_all.append(history)
    axs[i][0].plot(history.epoch, history.history["loss"], label="Train loss")
    axs[i][0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    axs[i][1].plot(history.epoch, history.history["acc"], label="Train accuracy")
    axs[i][1].plot(history.epoch, history.history["val_acc"], label="Validation accuracy")
    axs[i][2].plot(history.epoch, history.history["mean_iou"], label="Train iou")
    axs[i][2].plot(history.epoch, history.history["val_mean_iou"], label="Validation iou")

In [ ]:
plt.show()

# Predit with K-folds

In [ ]:
def predit_with_kfolds(K_flods,x_img,x_depth):
    preds_valid_all = []
    for i in range(K_flods):
        model_flods = load_model("trained_models/%dth_flod.model"%i, custom_objects={'mean_iou': mean_iou})
    #     model.append(model_flods)
        #此处的validation为第0组flod
        preds_valid_flods = model_flods.predict({'img': x_img, 
                                'depth': x_depth}).reshape(-1, img_size_target, img_size_target)
        preds_valid_flods = np.array([downsample(x) for x in preds_valid_flods])
        preds_valid_all.append(preds_valid_flods)
    preds_valid = (preds_valid_all[0]+preds_valid_all[1]+preds_valid_all[2]+preds_valid_all[3]+preds_valid_all[4])/5
    return preds_valid


In [ ]:
valid_x = np.array(my_test_pd.images_d.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 2)
valid_y = np.array(my_test_pd.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)

preds_valid = predit_with_kfolds(K_flods,valid_x[..., :1],valid_x[:, 60:60+DPT_SIZE, 60:60+DPT_SIZE, 1:])
# preds_valid_all = []
# for i in range(K_flods):
#     model_flods = load_model("trained_models/%dth_flod.model"%i, custom_objects={'mean_iou': mean_iou})
# #     model.append(model_flods)
#     #此处的validation为第0组flod
#     preds_valid_flods = model_flods.predict({'img': x_valid[0][..., :1], 
#                             'depth': x_valid[0][:, 60:60+DPT_SIZE, 60:60+DPT_SIZE, 1:]}).reshape(-1, img_size_target, img_size_target)
#     preds_valid_flods = np.array([downsample(x) for x in preds_valid_flods])
#     preds_valid_all.append(preds_valid_flods)
# preds_valid = (preds_valid_all[0]+preds_valid_all[1]+preds_valid_all[2]+preds_valid_all[3]+preds_valid_all[4])/5
y_valid = np.array([downsample(x) for x in valid_y])

In [ ]:
preds_valid.shape

In [ ]:
# max_images = 60
# grid_width = 15
# grid_height = int(max_images / grid_width)
# fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
# for i, idx in enumerate(ids_valid[60:60+max_images]):
#     img = train_df.loc[idx].images
#     mask = train_df.loc[idx].masks
#     pred = preds_valid[i]
#     ax = axs[int(i / grid_width), i % grid_width]
#     ax.imshow(img, cmap="Greys")
#     ax.imshow(mask, alpha=0.3, cmap="Greens")
#     ax.imshow(pred, alpha=0.3, cmap="OrRd")
#     ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
#     ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
#     ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
#     ax.set_yticklabels([])
#     ax.set_xticklabels([])
# plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

In [ ]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union
    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)


In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(y_valid, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
plt.show()


# Another sanity check with adjusted threshold

Again some sample images with the adjusted threshold.


In [ ]:
# max_images = 60
# grid_width = 15
# grid_height = int(max_images / grid_width)
# fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
# for i, idx in enumerate(ids_valid[:max_images]):
#     img = train_df.loc[idx].images
#     mask = train_df.loc[idx].masks
#     pred = preds_valid[i]
#     ax = axs[int(i / grid_width), i % grid_width]
#     ax.imshow(img, cmap="Greys")
#     ax.imshow(mask, alpha=0.3, cmap="Greens")
#     ax.imshow(np.array(np.round(pred > threshold_best), dtype=np.float32), alpha=0.3, cmap="OrRd")
#     ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
#     ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
#     ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
#     ax.set_yticklabels([])
#     ax.set_xticklabels([])
# plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")


# Submission

Load, predict and submit the test image predictions.


In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [ ]:
x_test = np.array([upsample(np.array(load_img("test/images/{}.png".format(idx), grayscale=True))) / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)
# Create depth layer
del x_train,y_train,x_valid,y_valid
x_test_d = [np.ones((DPT_SIZE,DPT_SIZE,1)) * (test_df.loc[i]["z"] / MAX_DEPTH)
                     for i in tqdm_notebook(test_df.index)] 
x_test_d = np.array(x_test_d).reshape(-1, DPT_SIZE, DPT_SIZE, 1)
x_test_d.shape

preds_test = predit_with_kfolds(K_flods,x_test,x_test_d)
# preds_test = model.predict({'img': x_test, 'depth': x_test_d})
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > 
            threshold_best)) for i, idx in enumerate(tqdm_notebook(test_df.index.values))}

sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')